In [5]:
import re
import json

code = """

const AAA: int = 1

def func(a: int) -> int:
    a = a + 1
    return a

def main() -> None:
    var a: int = 2
    var b: string = "eiwf23h32ur"
    
    var c: int = 12
    del a
    var a: int = 1
    
    c = a + 10

    var x: int = 0
    x = func(c) + 1


    var v: int = (c + a) // 10

    var xx: int = 10 + 1
    xx = 1 + 10

    del xx


    var i: int = 0
    var sum: int = 0

    while i < 10:
        sum = sum + i
        i = i + 1
    
    for j in range(0, 10):
        sum = sum + j
"""



# code = """
# def main() -> None:
#     var a: int = 0
#     var x: int = 100
#     a = (1 / 1) + (1 * x) / 10
# """




# code = """
# def main() -> None:
#     def func() -> None:
#         func()
# """


# code = """
# def main() -> None:
#     def func(a: int) -> None:
#         func(a)
# """


# code = """
# def main() -> None:
#     var a: int = 10
#     del a
# """



# code = """
# def main() -> None:
#     var a: int = 10
#     del a
# """




# code = """
# def main() -> None:
#     var i: int = 0
#     var sum: int = 0
    
#     while i < 10:
#         sum = sum + i
#         i = i + 1
    
#     for j in range(0, 10):
#         sum = sum + j
# """



KEYS = ["const", "var", "def", "del", "return", "while", "for"]
DATA_TYPES = ["int", "string", "None"]

class SymbolTable:
    def __init__(self):
        self.symbols = {}
    
    def add_symbol(self, name, key, var_type, value=None, pointer=None, is_constant=False):
        symbol_id = name
        
        self.symbols[symbol_id] = {
            "name": name,
            "key": key,
            "type": var_type,
            "value": value,
            "pointer": pointer or f"{key}_{name}_ptr_{id(self)}",
            "id": symbol_id
        }
        
        if is_constant:
            self.symbols[symbol_id]["key"] = "const"
        
        return symbol_id

    def get_symbol(self, name):
        return self.symbols.get(name)

    def update_symbol(self, name, updates):
        if name in self.symbols:
            self.symbols[name].update(updates)
            return True
        return False
    
    def delete_symbol(self, name):
        if name in self.symbols:
            del self.symbols[name]
            return True
        return False


class Parser:
    def __init__(self):
        self.scopes = []  # Список всех областей видимости
        self.scope_stack = []  # Стек текущих областей видимости
        self.symbol_counter = 0
        self.current_indent = 0
        self.indent_size = None  # Размер отступа (4 пробела или 1 таб)
        self.indent_char = None  # Тип отступа ('space' или 'tab')
    
    def detect_indent_type(self, line: str):
        if not line.startswith((' ', '\t')):
            return None
        
        first_char = line[0]
        if first_char == '\t':
            return ('tab', 1)
        elif first_char == ' ':
            space_count = 0
            for char in line:
                if char == ' ':
                    space_count += 1
                else:
                    break
            
            common_indents = [2, 4, 8]
            for indent in common_indents:
                if space_count % indent == 0:
                    return ('space', indent)
            
            return ('space', space_count)
        
        return None
    
    def analyze_indent_pattern(self, lines: list) -> tuple:
        tab_lines = 0
        space_lines = 0
        space_counts = {}
        
        for line in lines:
            if line.startswith('\t'):
                tab_lines += 1
            elif line.startswith(' '):
                space_lines += 1
                space_count = 0
                for char in line:
                    if char == ' ':
                        space_count += 1
                    else:
                        break
                if space_count > 0:
                    space_counts[space_count] = space_counts.get(space_count, 0) + 1
        
        common_space_count = None
        if space_counts:
            common_space_count = max(space_counts.items(), key=lambda x: x[1])[0]
        
        if tab_lines > 0 and space_lines > 0:
            if tab_lines > space_lines:
                return ('tab', 1)
            else:
                if common_space_count:
                    for indent in [2, 4, 8]:
                        if common_space_count % indent == 0:
                            return ('space', indent)
                    return ('space', common_space_count)
        
        if tab_lines > 0:
            return ('tab', 1)
        
        if space_lines > 0 and common_space_count:
            for indent in [2, 4, 8]:
                if common_space_count % indent == 0:
                    return ('space', indent)
            return ('space', common_space_count)
        
        return ('space', 4)
    
    def handle_indent_change(self, indent: int):
        if indent > self.current_indent:
            self.current_indent = indent
        elif indent < self.current_indent:
            while self.current_indent > indent and len(self.scope_stack) > 1:
                self.scope_stack.pop()
                self.current_indent -= 1
    
    def generate_pointer(self, prefix):
        self.symbol_counter += 1
        return f"{prefix}_ptr_{self.symbol_counter:03d}"
    
    def parse_line(self, line: str, scope: dict, all_lines: list, current_index: int, indent: int):
        parsed = False
        
        for key in KEYS:
            if line.startswith(key + " ") or line == key:
                if key == "const":
                    parsed = self.parse_const(line, scope)
                elif key == "var":
                    parsed = self.parse_var(line, scope)
                elif key == "def":
                    parsed = self.parse_function_declaration(line, scope, all_lines, current_index)
                elif key == "del":
                    parsed = self.parse_delete(line, scope)
                elif key == "return":
                    parsed = self.parse_return(line, scope)
                elif key == "while":
                    parsed = self.parse_while_loop(line, scope, all_lines, current_index, indent)
                elif key == "for":
                    parsed = self.parse_for_loop(line, scope, all_lines, current_index, indent)
                break
        
        if not parsed:
            if re.match(r'[a-zA-Z_][a-zA-Z0-9_]*\s*\(', line) and 'var ' in line:
                parsed = self.parse_function_call_assignment(line, scope)
            elif re.match(r'[a-zA-Z_][a-zA-Z0-9_]*\s*\(', line):
                parsed = self.parse_function_call(line, scope)
            elif "=" in line and not any(line.startswith(k + " ") for k in ["const", "var", "def"]):
                parsed = self.parse_assignment(line, scope)
            elif "+=" in line or "-=" in line or "*=" in line or "/=" in line:
                parsed = self.parse_augmented_assignment(line, scope)
        
        return parsed
    
    def parse_code(self, code: str):
        code = re.sub(r"#.*", "", code)
        code = re.sub(r"'''.*?'''", "", code, flags=re.DOTALL)
        code = re.sub(r'""".*?"""', "", code, flags=re.DOTALL)
        
        lines = code.split("\n")
        
        if any(line.startswith((' ', '\t')) for line in lines if line.strip()):
            self.indent_char, self.indent_size = self.analyze_indent_pattern(lines)
        
        global_scope = {
            "level": 0,
            "type": "module",
            "parent_scope": None,
            "local_variables": [],
            "graph": [],
            "symbol_table": SymbolTable()
        }
        self.scopes.append(global_scope)
        self.scope_stack = [global_scope]
        self.current_indent = 0
        
        i = 0
        while i < len(lines):
            line = lines[i]
            
            if not line.strip():
                i += 1
                continue
            
            indent = self.calculate_indent_level(line)
            line_content = line.strip()
            
            # Получаем текущую область видимости
            current_scope = self.get_current_scope_for_indent(indent)
            
            if line_content:
                i = self.parse_line_with_context(line_content, current_scope, lines, i, indent)
            else:
                i += 1
        
        for scope in self.scopes:
            scope["symbol_table"] = scope["symbol_table"].symbols
        
        return self.scopes
    

    def get_current_scope_for_indent(self, indent: int):
        """Возвращает область видимости для заданного уровня отступа"""
        # Если отступ 0 - возвращаем глобальную область
        if indent == 0:
            # Находим глобальную область в стеке
            for scope in self.scope_stack:
                if scope["level"] == 0:
                    return scope
            return self.scope_stack[0] if self.scope_stack else None
        
        # Находим область с нужным уровнем
        for scope in reversed(self.scope_stack):
            if scope["level"] <= indent:
                return scope
        
        # Если не нашли, возвращаем последнюю область
        return self.scope_stack[-1] if self.scope_stack else None

    
    def calculate_indent_level(self, line: str) -> int:
        if not line.startswith((' ', '\t')):
            return 0
        
        if self.indent_size is None:
            indent_info = self.detect_indent_type(line)
            if indent_info:
                self.indent_char, self.indent_size = indent_info
        
        if self.indent_char == 'tab':
            tab_count = 0
            for char in line:
                if char == '\t':
                    tab_count += 1
                else:
                    break
            return tab_count
        elif self.indent_char == 'space':
            space_count = 0
            for char in line:
                if char == ' ':
                    space_count += 1
                else:
                    break
            return space_count // self.indent_size if self.indent_size > 0 else 0
        
        return 0
    
    def get_current_scope(self, indent):
        """Определяет текущий scope на основе отступа"""
        if indent == 0:
            return self.scopes[0]  # Глобальная область
        
        # Ищем самую глубокую функцию
        for scope in reversed(self.scopes):
            if scope["type"] == "function":
                return scope
        
        return self.scopes[0]
    
    def parse_line(self, line: str, scope: dict, all_lines: list, current_index: int, indent: int):
        parsed = False
        
        for key in KEYS:
            if line.startswith(key + " ") or line == key:
                if key == "const":
                    parsed = self.parse_const(line, scope)
                elif key == "var":
                    parsed = self.parse_var(line, scope)
                elif key == "def":
                    parsed = self.parse_function_declaration(line, scope, all_lines, current_index)
                elif key == "del":
                    parsed = self.parse_delete(line, scope)
                elif key == "return":
                    parsed = self.parse_return(line, scope)
                elif key == "while":
                    parsed = self.parse_while_loop(line, scope, all_lines, current_index, indent)
                elif key == "for":
                    parsed = self.parse_for_loop(line, scope, all_lines, current_index, indent)
                break
        
        if not parsed:
            if re.match(r'[a-zA-Z_][a-zA-Z0-9_]*\s*\(', line) and 'var ' in line:
                parsed = self.parse_function_call_assignment(line, scope)
            elif re.match(r'[a-zA-Z_][a-zA-Z0-9_]*\s*\(', line):
                parsed = self.parse_function_call(line, scope)
            elif "=" in line and not any(line.startswith(k + " ") for k in ["const", "var", "def"]):
                parsed = self.parse_assignment(line, scope)
            elif "+=" in line or "-=" in line or "*=" in line or "/=" in line:
                parsed = self.parse_augmented_assignment(line, scope)
        
        # Всегда возвращаем следующий индекс
        return current_index + 1
    
    def parse_line_with_context(self, line: str, scope: dict, all_lines: list, current_index: int, indent: int):
        """Парсит строку с учетом контекста и возвращает новый индекс"""
        # Проверяем, является ли это объявлением функции
        if line.startswith("def "):
            return self.parse_function_declaration(line, scope, all_lines, current_index)
        
        # Проверяем циклы
        elif line.startswith("while "):
            return self.parse_while_loop(line, scope, all_lines, current_index, indent)
        
        elif line.startswith("for "):
            return self.parse_for_loop(line, scope, all_lines, current_index, indent)
        
        # Остальные случаи
        else:
            return self.parse_line(line, scope, all_lines, current_index, indent)
    
    def parse_global_line(self, line: str, scope: dict, all_lines: list, current_index: int):
        """Парсит строку в глобальной области видимости"""
        if not line:
            return
        
        for key in KEYS:
            if line.startswith(key + " ") or line == key:
                if key == "const":
                    self.parse_const(line, scope)
                elif key == "var":
                    self.parse_var(line, scope)
                elif key == "def":
                    self.parse_function_declaration(line, scope, all_lines, current_index)
                return
        
        # В глобальной области только объявления
        print(f"Warning: Unexpected line in global scope: {line}")
    
    def parse_function_line(self, line: str, scope: dict):
        """Парсит строку внутри функции"""
        if not line:
            return
        
        parsed = False
        
        for key in KEYS:
            if line.startswith(key + " ") or line == key:
                if key == "const":
                    parsed = self.parse_const(line, scope)
                elif key == "var":
                    parsed = self.parse_var(line, scope)
                elif key == "def":
                    # Вложенные функции пока не поддерживаем
                    parsed = False
                elif key == "del":
                    parsed = self.parse_delete(line, scope)
                elif key == "return":
                    parsed = self.parse_return(line, scope)
                break
        
        if not parsed:
            if re.match(r'[a-zA-Z_][a-zA-Z0-9_]*\s*\(', line) and 'var ' in line:
                parsed = self.parse_function_call_assignment(line, scope)
            elif re.match(r'[a-zA-Z_][a-zA-Z0-9_]*\s*\(', line):
                parsed = self.parse_function_call(line, scope)
            elif "=" in line and not any(line.startswith(k + " ") for k in ["const", "var", "def"]):
                parsed = self.parse_assignment(line, scope)
            elif "+=" in line or "-=" in line or "*=" in line or "/=" in line:
                parsed = self.parse_augmented_assignment(line, scope)
    
    def parse_function_declaration(self, line: str, parent_scope: dict, all_lines: list, current_index: int):
        """Обрабатывает объявление функции"""
        pattern = r"def\s+([a-zA-Z_][a-zA-Z0-9_]*)\s*\((.*?)\)\s*(?:->\s*([a-zA-Z_][a-zA-Z0-9_]*))?\s*:"
        match = re.match(pattern, line)
        
        if not match:
            return current_index + 1
        
        func_name, params_str, return_type = match.groups()
        return_type = return_type if return_type else "None"
        
        # Парсим параметры
        parameters = []
        if params_str.strip():
            param_pattern = r'([a-zA-Z_][a-zA-Z0-9_]*)\s*:\s*([a-zA-Z_][a-zA-Z0-9_]*)'
            params = re.findall(param_pattern, params_str)
            for param_name, param_type in params:
                parameters.append({
                    "name": param_name,
                    "type": param_type
                })
        
        # Определяем уровень вложенности функции
        parent_level = parent_scope["level"]
        func_level = parent_level + 1
        
        # Добавляем функцию в таблицу символов родительской области
        symbol_id = parent_scope["symbol_table"].add_symbol(
            name=func_name,
            key="function",
            var_type="function",
            pointer=self.generate_pointer(f"func_{func_name}")
        )
        
        parent_scope["graph"].append({
            "node": "function_declaration",
            "content": line,
            "function_name": func_name,
            "symbol_id": symbol_id,
            "parameters": parameters,
            "return_type": return_type,
            "body_level": func_level
        })
        
        # Создаем новую область видимости для функции
        func_scope = {
            "level": func_level,
            "type": "function",
            "parent_scope": parent_scope["level"],  # Ссылка на родительскую область
            "function_name": func_name,
            "parameters": parameters,
            "return_type": return_type,
            "local_variables": [],
            "graph": [],
            "symbol_table": SymbolTable(),
            "return_info": {
                "has_return": False,
                "return_value": None,
                "return_type": return_type
            }
        }
        
        # Добавляем параметры в таблицу символов функции
        for param in parameters:
            func_scope["symbol_table"].add_symbol(
                name=param["name"],
                key="var",
                var_type=param["type"]
            )
            func_scope["local_variables"].append(param["name"])
        
        # Добавляем scope функции в общий список и в стек
        self.scopes.append(func_scope)
        self.scope_stack.append(func_scope)  # Добавляем в стек для обработки тела функции

        body_start = current_index + 1
        body_end = self.find_indented_block_end(all_lines, body_start)
        
        return body_end
    
    def find_indented_block_end(self, lines: list, start_index: int) -> int:
        """Находит конец блока с отступом"""
        if start_index >= len(lines):
            return start_index
        
        # Определяем отступ первой строки блока
        first_line = lines[start_index]
        base_indent = self.calculate_indent_level(first_line)
        
        i = start_index
        while i < len(lines):
            line = lines[i]
            if not line.strip():
                i += 1
                continue
            
            current_indent = self.calculate_indent_level(line)
            
            # Если отступ стал меньше или равен базовому (и строка не пустая)
            if current_indent <= base_indent:
                return i
            
            i += 1
        
        return len(lines)
    
    def parse_const(self, line: str, scope: dict):
        pattern = r"const\s+([a-zA-Z_][a-zA-Z0-9_]*)\s*:\s*([a-zA-Z_][a-zA-Z0-9_]*)\s*=\s*(.+)"
        match = re.match(pattern, line)
        
        if match:
            name, var_type, value = match.groups()
            value = self.clean_value(value)
            
            symbol_id = scope["symbol_table"].add_symbol(
                name=name,
                key="const",
                var_type=var_type,
                value=value,
                is_constant=True
            )
            
            scope["local_variables"].append(symbol_id)
            
            scope["graph"].append({
                "node": "declaration",
                "content": line,
                "symbols": [symbol_id],
                "operations": [
                    {"type": "NEW_CONST", "target": symbol_id, "const_type": var_type},
                    {"type": "ASSIGN", "target": symbol_id, "value": value}
                ]
            })
            
            return True
        return False
    
    def parse_var(self, line: str, scope: dict):
        pattern = r"var\s+([a-zA-Z_][a-zA-Z0-9_]*)\s*:\s*([a-zA-Z_][a-zA-Z0-9_]*)\s*=\s*(.+)"
        match = re.match(pattern, line)
        
        if match:
            name, var_type, value = match.groups()
            value = self.clean_value(value)
            
            symbol_id = scope["symbol_table"].add_symbol(
                name=name,
                key="var",
                var_type=var_type,
                value=value
            )
            
            scope["local_variables"].append(symbol_id)
            
            scope["graph"].append({
                "node": "declaration",
                "content": line,
                "symbols": [symbol_id],
                "operations": [
                    {"type": "NEW_VAR", "target": symbol_id, "var_type": var_type},
                    {"type": "ASSIGN", "target": symbol_id, "value": value}
                ]
            })
            
            return True
        return False
    
    def parse_delete(self, line: str, scope: dict):
        pattern = r"del\s+([a-zA-Z_][a-zA-Z0-9_]*)"
        match = re.match(pattern, line)
        
        if not match:
            return False
        
        name = match.group(1)
        
        deleted = scope["symbol_table"].delete_symbol(name)
        
        if deleted:
            # Также удаляем из local_variables если есть
            if name in scope["local_variables"]:
                scope["local_variables"].remove(name)
            
            scope["graph"].append({
                "node": "delete",
                "content": line,
                "symbols": [name],
                "operations": [
                    {"type": "DELETE", "target": name}
                ]
            })
        
        return deleted
    
    def parse_return(self, line: str, scope: dict):
        """Парсит оператор return"""
        pattern = r"return\s+(.+)"
        match = re.match(pattern, line)
        
        if not match:
            return False
        
        value = match.group(1).strip()
        
        dependencies = []
        var_pattern = r'([a-zA-Z_][a-zA-Z0-9_]*)'
        vars_in_value = re.findall(var_pattern, value)
        for var in vars_in_value:
            if var not in KEYS and var not in DATA_TYPES:
                dependencies.append(var)
        
        scope["graph"].append({
            "node": "return",
            "content": line,
            "symbols": [value] if value.isalpha() else [],
            "operations": [
                {"type": "RETURN", "value": value}
            ],
            "dependencies": dependencies
        })
        
        if "return_info" in scope:
            scope["return_info"]["has_return"] = True
            scope["return_info"]["return_value"] = value
        
        return True
    
    
    
    def parse_assignment(self, line: str, scope: dict):
        pattern = r"([a-zA-Z_][a-zA-Z0-9_]*)\s*=\s*(.+)"
        match = re.match(pattern, line)
        
        if not match:
            return False
        
        name, expression = match.groups()
        
        symbol = scope["symbol_table"].get_symbol(name)
        if not symbol:
            return False
        
        scope["symbol_table"].update_symbol(name, {"value": expression})
        
        operations = []
        dependencies = []
        
        # Сначала проверяем, есть ли операторы в выражении
        if self.contains_operator(expression):
            # Используем улучшенный парсер сложных выражений
            self.parse_complex_expression(name, expression, operations, dependencies, scope)
        else:
            # Проверяем унарные операции
            expression_stripped = expression.strip()
            unary_ops = {
                '-': 'NEGATE',
                '+': 'UNARY_PLUS', 
                '~': 'BITWISE_NOT'
            }
            
            unary_found = False
            for op_symbol, op_type in unary_ops.items():
                if expression_stripped.startswith(op_symbol) and len(expression_stripped) > len(op_symbol):
                    next_char = expression_stripped[len(op_symbol)]
                    # Проверяем, что это унарная операция (не часть числа)
                    if next_char.isalnum() or next_char == '(':
                        value = expression_stripped[len(op_symbol):].strip()
                        
                        operations.append({
                            "type": "UNARY_OPERATION",
                            "target": name,
                            "operator": op_type,
                            "operator_symbol": op_symbol,
                            "value": value
                        })
                        
                        value_clean = value.strip('() ')
                        if value_clean and value_clean.isalpha() and value_clean not in KEYS and value_clean not in DATA_TYPES:
                            dependencies.append(value_clean)
                        
                        unary_found = True
                        break
            
            if not unary_found:
                # Простое присваивание
                operations.append({
                    "type": "ASSIGN",
                    "target": name,
                    "value": self.clean_value(expression)
                })
                
                # Проверяем зависимости для простых присваиваний
                clean_expr = expression.strip('() ')
                if clean_expr and clean_expr.isalpha() and clean_expr not in KEYS and clean_expr not in DATA_TYPES:
                    dependencies.append(clean_expr)
        
        # Обновляем значение переменной
        scope["symbol_table"].add_symbol(
            name=name,
            key="var",
            var_type=symbol["type"],
            value=expression
        )
        
        scope["graph"].append({
            "node": "assignment",
            "content": line,
            "symbols": [name],
            "operations": operations,
            "dependencies": dependencies
        })
        
        return True
    
    
    
    def parse_augmented_assignment(self, line: str, scope: dict):
        """Парсит составные операции присваивания"""
        pattern = r"([a-zA-Z_][a-zA-Z0-9_]*)\s*(\+=|-=|\*=|/=|//=|\%=|\*\*=|>>=|<<=|&=|\|=|\^=)\s*(.+)"
        match = re.match(pattern, line)
        
        if not match:
            return False
        
        name, operator, value = match.groups()
        
        symbol = scope["symbol_table"].get_active_symbol(name)
        if not symbol:
            return False
        
        # Определяем тип операции
        operator_map = {
            '+=': 'ADD',
            '-=': 'SUBTRACT',
            '*=': 'MULTIPLY',
            '/=': 'DIVIDE',
            '//=': 'INTEGER_DIVIDE',
            '%=': 'MODULO',
            '**=': 'POWER',
            '>>=': 'RIGHT_SHIFT',
            '<<=': 'LEFT_SHIFT',
            '&=': 'BITWISE_AND',
            '|=': 'BITWISE_OR',
            '^=': 'BITWISE_XOR'
        }
        
        op_type = operator_map.get(operator, 'UNKNOWN_AUGMENTED')
        
        operations = [{
            "type": "AUGMENTED_ASSIGN",
            "target": name,
            "operator": op_type,
            "operator_symbol": operator,
            "value": value
        }]
        
        dependencies = []
        if value.isalpha() and value not in KEYS and value not in DATA_TYPES:
            dependencies.append(value)
        
        # Обновляем значение переменной
        scope["symbol_table"].add_symbol(
            name=name,
            key="var",
            var_type=symbol["type"],
            value=f"{name} {operator} {value}"
        )
        
        scope["graph"].append({
            "node": "augmented_assignment",
            "content": line,
            "symbols": [name],
            "operations": operations,
            "dependencies": dependencies
        })
        
        return True

    def parse_expression(self, expression: str, target_var: str, scope: dict):
        """Парсит сложные выражения с несколькими операциями"""
        # Упрощенная версия - поддерживает только одну операцию
        # Для полной поддержки нужно реализовать парсер выражений с учетом приоритета
        
        operators = [
            ('**', 'POWER', 10),
            ('*', 'MULTIPLY', 9),
            ('/', 'DIVIDE', 9),
            ('//', 'INTEGER_DIVIDE', 9),
            ('%', 'MODULO', 9),
            ('+', 'ADD', 8),
            ('-', 'SUBTRACT', 8),
            ('<<', 'LEFT_SHIFT', 7),
            ('>>', 'RIGHT_SHIFT', 7),
            ('&', 'BITWISE_AND', 6),
            ('^', 'BITWISE_XOR', 5),
            ('|', 'BITWISE_OR', 4)
        ]
        
        # Ищем оператор с наивысшим приоритетом
        for op_symbol, op_type, priority in operators:
            if op_symbol in expression:
                parts = expression.split(op_symbol, 1)  # Разделяем только по первому вхождению
                if len(parts) == 2:
                    left, right = parts[0].strip(), parts[1].strip()
                    
                    operations = [{
                        "type": "BINARY_OPERATION",
                        "target": target_var,
                        "operator": op_type,
                        "operator_symbol": op_symbol,
                        "left": left,
                        "right": right
                    }]
                    
                    dependencies = []
                    if left.isalpha() and left not in KEYS and left not in DATA_TYPES:
                        dependencies.append(left)
                    if right.isalpha() and right not in KEYS and right not in DATA_TYPES:
                        dependencies.append(right)
                    
                    return operations, dependencies
        
        # Если операций нет - простое присваивание
        return [{
            "type": "ASSIGN",
            "target": target_var,
            "value": self.clean_value(expression)
        }], []
    
    def parse_complex_expression(self, target: str, expression: str, operations: list, dependencies: list, scope: dict):
        """Разбирает сложные выражения с несколькими операторами и скобками"""
        expression = expression.strip()
        
        # Убираем внешние скобки, если выражение полностью в них
        while self.is_fully_parenthesized(expression):
            expression = expression[1:-1].strip()
        
        # Проверяем, содержит ли выражение операторы
        if not self.contains_operator(expression):
            # Нет операторов - это простое значение или переменная
            clean_expr = expression.strip('() ')
            if clean_expr and clean_expr.isalpha() and clean_expr not in KEYS and clean_expr not in DATA_TYPES:
                dependencies.append(clean_expr)
            
            operations.append({
                "type": "ASSIGN",
                "target": target,
                "value": self.clean_value(expression)
            })
            return
        
        # Находим оператор с наименьшим приоритетом
        operator_info = self.find_lowest_priority_operator(expression)
        
        if not operator_info:
            # Если не нашли оператор, возможно выражение в скобках содержит операторы
            # Попробуем разобрать как есть
            clean_expr = expression.strip('() ')
            if clean_expr:
                temp_var = f"{target}_inner"
                self.parse_complex_expression(temp_var, clean_expr, operations, dependencies, scope)
                operations.append({
                    "type": "ASSIGN",
                    "target": target,
                    "value": temp_var
                })
            return
        
        op_symbol, op_type, op_index = operator_info
        left = expression[:op_index].strip()
        right = expression[op_index + len(op_symbol):].strip()
        
        # Добавляем основную операцию
        operations.append({
            "type": "BINARY_OPERATION",
            "target": target,
            "operator": op_type,
            "operator_symbol": op_symbol,
            "left": left,
            "right": right
        })
        
        # Вспомогательная функция для разбора части выражения
        def parse_subexpression(subexpr: str, side: str):
            subexpr = subexpr.strip()
            if not subexpr:
                return
            
            # Убираем внешние скобки
            while self.is_fully_parenthesized(subexpr):
                subexpr = subexpr[1:-1].strip()
            
            if self.contains_operator(subexpr):
                # Создаем временную переменную для подвыражения
                temp_var = f"{target}_{side}_{len(operations)}"
                self.parse_complex_expression(temp_var, subexpr, operations, dependencies, scope)
                # Обновляем ссылку в основной операции
                for op in operations:
                    if op.get("target") == target and op.get("type") == "BINARY_OPERATION":
                        if side == "left":
                            op["left"] = temp_var
                        else:
                            op["right"] = temp_var
            else:
                # Проверяем зависимости
                clean_subexpr = subexpr.strip('() ')
                if clean_subexpr and clean_subexpr.isalpha() and clean_subexpr not in KEYS and clean_subexpr not in DATA_TYPES:
                    dependencies.append(clean_subexpr)
        
        # Рекурсивно разбираем левую и правую части
        parse_subexpression(left, "left")
        parse_subexpression(right, "right")
    
    def is_fully_parenthesized(self, expression: str) -> bool:
        """Проверяет, полностью ли выражение заключено в скобки"""
        if not expression.startswith('(') or not expression.endswith(')'):
            return False
        
        # Проверяем баланс скобок
        balance = 0
        for i, char in enumerate(expression):
            if char == '(':
                balance += 1
            elif char == ')':
                balance -= 1
                # Если баланс стал 0 до конца строки, это не полное обрамление
                if balance == 0 and i < len(expression) - 1:
                    return False
        
        return balance == 0
    
    def find_lowest_priority_operator(self, expression: str):
        """Находит оператор с наименьшим приоритетом вне скобок"""
        # Приоритет операций (от низшего к высшему)
        operator_levels = [
            # Уровень 1 (наименьший приоритет)
            [('|', 'BITWISE_OR')],
            # Уровень 2
            [('^', 'BITWISE_XOR')],
            # Уровень 3
            [('&', 'BITWISE_AND')],
            # Уровень 4
            [('<<', 'LEFT_SHIFT'), ('>>', 'RIGHT_SHIFT')],
            # Уровень 5
            [('+', 'ADD'), ('-', 'SUBTRACT')],
            # Уровень 6
            [('*', 'MULTIPLY'), ('/', 'DIVIDE'), ('//', 'INTEGER_DIVIDE'), ('%', 'MODULO')],
            # Уровень 7 (наивысший приоритет)
            [('**', 'POWER')]
        ]
        
        # Ищем операторы от низшего приоритета к высшему
        for level in operator_levels:
            for op_symbol, op_type in level:
                # Ищем оператор вне скобок
                index = self.find_operator_outside_parentheses(expression, op_symbol)
                if index != -1:
                    return (op_symbol, op_type, index)
        
        return None
    
    def find_operator_outside_parentheses(self, expression: str, operator: str) -> int:
        """Находит позицию оператора вне скобок"""
        balance = 0
        i = 0
        
        while i < len(expression):
            char = expression[i]
            
            if char == '(':
                balance += 1
            elif char == ')':
                balance -= 1
            elif balance == 0:  # Мы вне скобок
                # Проверяем, является ли текущая позиция началом оператора
                if expression[i:i+len(operator)] == operator:
                    # Проверяем, что это действительно оператор, а не часть идентификатора или числа
                    before_ok = i == 0 or not expression[i-1].isalnum()
                    after_ok = i + len(operator) >= len(expression) or not expression[i+len(operator)].isalnum()
                    
                    if before_ok and after_ok:
                        return i
            
            i += 1
        
        return -1
    
    def contains_operator(self, expression: str) -> bool:
        """Проверяет, содержит ли выражение какой-либо оператор"""
        expression = expression.strip()
        
        # Сначала убираем внешние скобки
        while self.is_fully_parenthesized(expression):
            expression = expression[1:-1].strip()
        
        operators = ['+', '-', '*', '/', '//', '%', '**', '>>', '<<', '&', '|', '^']
        
        balance = 0
        for i, char in enumerate(expression):
            if char == '(':
                balance += 1
            elif char == ')':
                balance -= 1
            elif balance == 0:  # Мы вне скобок
                for op in operators:
                    if expression[i:i+len(op)] == op:
                        # Проверяем контекст
                        before_ok = i == 0 or not expression[i-1].isalnum()
                        after_ok = i + len(op) >= len(expression) or not expression[i+len(op)].isalnum()
                        
                        if before_ok and after_ok:
                            return True
        
        return False
    
    def parse_function_call(self, line: str, scope: dict):
        """Парсит вызов функции"""
        pattern = r"([a-zA-Z_][a-zA-Z0-9_]*)\s*\((.*?)\)"
        match = re.match(pattern, line)
        
        if not match:
            return False
        
        func_name, args_str = match.groups()
        args = []
        if args_str.strip():
            args = [arg.strip() for arg in args_str.split(',')]
        
        operations = [{
            "type": "FUNCTION_CALL",
            "function": func_name,
            "arguments": args
        }]
        
        dependencies = []
        for arg in args:
            if arg.isalpha() and arg not in KEYS and arg not in DATA_TYPES:
                dependencies.append(arg)
        
        scope["graph"].append({
            "node": "function_call",
            "content": line,
            "function": func_name,
            "arguments": args,
            "operations": operations,
            "dependencies": dependencies
        })
        
        return True
    
    def parse_function_call_assignment(self, line: str, scope: dict):
        """Парсит присваивание результата вызова функции"""
        pattern = r"var\s+([a-zA-Z_][a-zA-Z0-9_]*)\s*:\s*([a-zA-Z_][a-zA-Z0-9_]*)\s*=\s*([a-zA-Z_][a-zA-Z0-9_]*)\s*\((.*?)\)"
        match = re.match(pattern, line)
        
        if not match:
            return False
        
        var_name, var_type, func_name, args_str = match.groups()
        args = []
        if args_str.strip():
            args = [arg.strip() for arg in args_str.split(',')]
        
        # Добавляем переменную
        symbol_id = scope["symbol_table"].add_symbol(
            name=var_name,
            key="var",
            var_type=var_type
        )
        
        scope["local_variables"].append(symbol_id)
        
        operations = [
            {"type": "NEW_VAR", "target": var_name, "var_type": var_type},
            {"type": "FUNCTION_CALL_ASSIGN", "function": func_name, "arguments": args, "target": var_name}
        ]
        
        dependencies = []
        for arg in args:
            if arg.isalpha() and arg not in KEYS and arg not in DATA_TYPES:
                dependencies.append(arg)
        
        scope["graph"].append({
            "node": "function_call_assignment",
            "content": line,
            "symbols": [var_name],
            "function": func_name,
            "arguments": args,
            "operations": operations,
            "dependencies": dependencies
        })
        
        return True
    
    def parse_condition(self, condition: str) -> dict:
        """Парсит условие для циклов и if"""
        operators = ['<', '>', '<=', '>=', '==', '!=', 'and', 'or']
        
        # Простая реализация - ищем операторы сравнения
        for op in operators:
            if op in condition:
                parts = condition.split(op, 1)
                if len(parts) == 2:
                    left, right = parts[0].strip(), parts[1].strip()
                    return {
                        "type": "COMPARISON",
                        "operator": op,
                        "left": left,
                        "right": right
                    }
        
        # Если не нашли оператор сравнения, предполагаем булевое выражение
        return {
            "type": "EXPRESSION",
            "value": condition
        }
    
    def parse_iterable(self, iterable_expr: str) -> dict:
        """Парсит итерируемое выражение для for цикла"""
        # Проверяем range вызов с 1, 2 или 3 аргументами
        range_pattern = r"range\s*\(\s*(.+?)\s*\)"
        range_match = re.match(range_pattern, iterable_expr)
        
        if range_match:
            args_str = range_match.group(1)
            # Разделяем аргументы по запятым, но учитываем возможные вложенные вызовы
            args = []
            current_arg = ""
            depth = 0  # Для отслеживания вложенных скобок
            
            for char in args_str:
                if char == '(':
                    depth += 1
                    current_arg += char
                elif char == ')':
                    depth -= 1
                    current_arg += char
                elif char == ',' and depth == 0:
                    args.append(current_arg.strip())
                    current_arg = ""
                else:
                    current_arg += char
            
            if current_arg:
                args.append(current_arg.strip())
            
            # Очищаем аргументы от лишних пробелов
            args = [arg.strip() for arg in args]
            
            # Определяем количество аргументов
            if len(args) == 1:
                # range(stop)
                return {
                    "type": "RANGE_CALL",
                    "function": "range",
                    "arguments": {
                        "start": "0",
                        "stop": args[0],
                        "step": "1"
                    }
                }
            elif len(args) == 2:
                # range(start, stop)
                return {
                    "type": "RANGE_CALL",
                    "function": "range",
                    "arguments": {
                        "start": args[0],
                        "stop": args[1],
                        "step": "1"
                    }
                }
            elif len(args) == 3:
                # range(start, stop, step)
                return {
                    "type": "RANGE_CALL",
                    "function": "range",
                    "arguments": {
                        "start": args[0],
                        "stop": args[1],
                        "step": args[2]
                    }
                }
            else:
                # Некорректное количество аргументов
                return {
                    "type": "RANGE_CALL",
                    "function": "range",
                    "arguments": args
                }
        
        # Другие итерируемые объекты
        return {
            "type": "ITERABLE",
            "expression": iterable_expr
        }
    
    def find_loop_body_end(self, lines: list, start_index: int, base_indent: int) -> int:
        """Находит конец тела цикла"""
        if start_index >= len(lines):
            return start_index
        
        # Преобразуем base_indent в реальный отступ
        base_real_indent = base_indent + 1
        
        i = start_index
        while i < len(lines):
            line = lines[i]
            if not line.strip():
                i += 1
                continue
            
            current_indent = self.calculate_indent_level(line)
            
            # Если отступ стал меньше или равен базовому отступу (не включая увеличение для тела)
            if current_indent <= base_indent:
                return i
            
            i += 1
        
        return len(lines)
    
    def parse_while_loop(self, line: str, scope: dict, all_lines: list, current_index: int, indent: int):
        """Парсит while цикл"""
        # while условие:
        pattern = r"while\s+(.+?)\s*:"
        match = re.match(pattern, line)
        
        if not match:
            return current_index + 1
        
        condition = match.group(1).strip()
        
        # Парсим условие
        condition_ast = self.parse_condition(condition)
        
        # Находим тело цикла
        body_start = current_index + 1
        body_end = self.find_loop_body_end(all_lines, body_start, indent)
        
        # Создаем узел цикла
        loop_node = {
            "node": "while_loop",
            "content": line,
            "condition": condition_ast,
            "body_level": scope["level"] + 1,
            "body": []
        }
        
        scope["graph"].append(loop_node)
        
        # Сохраняем текущий стек и парсим тело цикла
        saved_stack = self.scope_stack.copy()
        saved_indent = self.current_indent
        
        # Создаем временную область для тела цикла
        loop_body_scope = {
            "level": scope["level"] + 1,
            "type": "loop_body",
            "parent_scope": scope["level"],
            "local_variables": [],
            "graph": [],
            "symbol_table": SymbolTable()
        }
        
        self.scopes.append(loop_body_scope)
        self.scope_stack.append(loop_body_scope)
        self.current_indent = indent + 1  # Увеличиваем отступ для тела цикла
        
        # Парсим тело цикла
        for j in range(body_start, body_end):
            body_line = all_lines[j]
            if body_line.strip():
                body_indent = self.calculate_indent_level(body_line)
                body_content = body_line.strip()
                self.parse_line(body_content, loop_body_scope, all_lines, j, body_indent)
        
        # Добавляем тело в узел цикла
        loop_node["body"] = loop_body_scope["graph"]
        
        # Восстанавливаем стек и отступ
        self.scope_stack = saved_stack
        self.current_indent = saved_indent

        body_start = current_index + 1
        body_end = self.find_indented_block_end(all_lines, body_start)

        
        return body_end
    
    def parse_for_loop(self, line: str, scope: dict, all_lines: list, current_index: int, indent: int):
        """Парсит for цикл"""
        # for переменная in итерируемый_объект:
        pattern = r"for\s+([a-zA-Z_][a-zA-Z0-9_]*)\s+in\s+(.+?)\s*:"
        match = re.match(pattern, line)
        
        if not match:
            return current_index + 1
        
        loop_var, iterable_expr = match.groups()
        loop_var = loop_var.strip()
        iterable_expr = iterable_expr.strip()
        
        # Парсим итерируемое выражение
        iterable_ast = self.parse_iterable(iterable_expr)
        
        # Находим тело цикла
        body_start = current_index + 1
        body_end = self.find_loop_body_end(all_lines, body_start, indent)
        
        # Создаем узел цикла
        loop_node = {
            "node": "for_loop",
            "content": line,
            "loop_variable": loop_var,
            "iterable": iterable_ast,
            "body_level": scope["level"] + 1,
            "body": []
        }
        
        scope["graph"].append(loop_node)
        
        # Добавляем переменную цикла в текущую область
        scope["symbol_table"].add_symbol(
            name=loop_var,
            key="var",
            var_type="int"  # Предполагаем int для range
        )
        scope["local_variables"].append(loop_var)
        
        # Сохраняем текущий стек и парсим тело цикла
        saved_stack = self.scope_stack.copy()
        saved_indent = self.current_indent
        
        # Создаем временную область для тела цикла
        loop_body_scope = {
            "level": scope["level"] + 1,
            "type": "loop_body",
            "parent_scope": scope["level"],
            "local_variables": [],
            "graph": [],
            "symbol_table": SymbolTable()
        }
        
        self.scopes.append(loop_body_scope)
        self.scope_stack.append(loop_body_scope)
        self.current_indent = indent + 1  # Увеличиваем отступ для тела цикла
        
        # Парсим тело цикла
        for j in range(body_start, body_end):
            body_line = all_lines[j]
            if body_line.strip():
                body_indent = self.calculate_indent_level(body_line)
                body_content = body_line.strip()
                self.parse_line(body_content, loop_body_scope, all_lines, j, body_indent)
        
        # Добавляем тело в узел цикла
        loop_node["body"] = loop_body_scope["graph"]
        
        # Восстанавливаем стек и отступ
        self.scope_stack = saved_stack
        self.current_indent = saved_indent

        body_start = current_index + 1
        body_end = self.find_indented_block_end(all_lines, body_start)

        # Пропускаем тело цикла
        return body_end
    
    def clean_value(self, value: str):
        """Очищает значение от лишних пробелов"""
        value = value.strip()
        if value.startswith('"') and value.endswith('"'):
            return value
        elif value.isdigit():
            return int(value)
        return value

def main():
    parser = Parser()
    result = parser.parse_code(code)
    
    json_output = json.dumps(result, indent=2, default=str)
    print(json_output)
    
    with open("parsed_code.json", "w") as f:
        f.write(json_output)
    
    return result

if __name__ == "__main__":
    main()

[
  {
    "level": 0,
    "type": "module",
    "parent_scope": null,
    "local_variables": [
      "AAA"
    ],
    "graph": [
      {
        "node": "declaration",
        "content": "const AAA: int = 1",
        "symbols": [
          "AAA"
        ],
        "operations": [
          {
            "type": "NEW_CONST",
            "target": "AAA",
            "const_type": "int"
          },
          {
            "type": "ASSIGN",
            "target": "AAA",
            "value": 1
          }
        ]
      },
      {
        "node": "function_declaration",
        "content": "def func(a: int) -> int:",
        "function_name": "func",
        "symbol_id": "func",
        "parameters": [
          {
            "name": "a",
            "type": "int"
          }
        ],
        "return_type": "int",
        "body_level": 1
      },
      {
        "node": "function_declaration",
        "content": "def main() -> None:",
        "function_name": "main",
        "symbol_id": "

const a: int = 1
var b: string = "hello"

var c: int = 1 + 2

In [ ]:
a = {
    "key": "const",
    "name": "a",
    "type": "int",
    "pointer": "efij213e32irh"
}

b = {
    "key": "var",
    "name": "b",
    "type": "string",
    "pointer": "askf21e128ndj"
}

In [ ]:
[
    {
        "1": {
            "local_variables": [a, b, c, d],
            "graph": [
                "NEW c",
                "SUMM c, (1, 2)"
            ]
        },
    }
]

const CONSTANT: int = 1

def main():
    var a: int = 2
    var b: string = "eiwf23h32ur"

    var c = 12
    del a
    var a: int = 1

    c = a + 10
    return c

const ANOTHER_CONST: string = "hello"